[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klavis-ai/klavis/blob/main/examples/openai/Use_Klavis_with_OpenAI.ipynb)


# OpenAI + Klavis AI Integration

This tutorial demonstrates how to use OpenAI function calling with Klavis MCP (Model Context Protocol) servers.


## Prerequisites

- **OpenAI API key** - Get at [openai.com](https://openai.com/)
- **Klavis API key** - Get at [klavis.ai](https://klavis.ai/)


In [1]:
# Install the required packages
%pip install -qU openai klavis requests


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
from openai import OpenAI
from klavis import Klavis
from klavis.types import McpServerName, ConnectionType, ToolFormat

# Set environment variables
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"  # Replace with your actual OpenAI API key
os.environ["KLAVIS_API_KEY"] = "YOUR_KLAVIS_API_KEY"  # Replace with your actual Klavis API key

## Case Study 1 : OpenAI + YouTube MCP Server

#### Step 1 - Create YouTube MCP Server using Klavis


In [3]:


klavis_client = Klavis(api_key=os.getenv("KLAVIS_API_KEY"))

youtube_mcp_instance = klavis_client.mcp_server.create_server_instance(
    server_name=McpServerName.YOUTUBE,
    user_id="1234",
    platform_name="Klavis",
    connection_type=ConnectionType.STREAMABLE_HTTP,
)

# print(f"🔗 YouTube MCP server created at: {youtube_mcp_instance.server_url}, and the instance id is {youtube_mcp_instance.instance_id}")


#### Step 2 - Create general method to use MCP Server with OpenAI

In [11]:
def openai_with_mcp_server(mcp_server_url: str, user_query: str):
    openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    messages = [
        {"role": "system", "content": "You are a helpful assistant. Use the available tools to answer the user's question."},
        {"role": "user", "content": f"{user_query}"}
    ]
    
    mcp_server_tools = klavis_client.mcp_server.list_tools(
        server_url=mcp_server_url,
        connection_type=ConnectionType.STREAMABLE_HTTP,
        format=ToolFormat.OPENAI,
    )
    
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=mcp_server_tools.tools
    )
    
    messages.append(response.choices[0].message)

    if response.choices[0].message.tool_calls:
        for tool_call in response.choices[0].message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"🔧 Calling: {function_name}, with args: {function_args}")
            
            result = klavis_client.mcp_server.call_tools(
                server_url=mcp_server_url,
                tool_name=function_name,
                tool_args=function_args,
                connection_type=ConnectionType.STREAMABLE_HTTP
            )
            
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(result)
            })
            
    final_response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    
    return final_response.choices[0].message.content

#### Step 3 - Summarize your favorite video! 

In [12]:
YOUTUBE_VIDEO_URL = "https://www.youtube.com/watch?v=LCEmiRjPEtQ"  # pick a video you like!

result = openai_with_mcp_server(
    mcp_server_url=youtube_mcp_instance.server_url, 
    user_query=f"Please provide a complete summary of this YouTube video with timestamp: {YOUTUBE_VIDEO_URL}"
)

print(result)

🔧 Calling: get_youtube_video_transcript, with args: {'url': 'https://www.youtube.com/watch?v=LCEmiRjPEtQ'}
The YouTube video titled "Andrej Karpathy: Software Is Changing (Again)" features a keynote by Andrej Karpathy at the AI Startup School in San Francisco. The video discusses the evolution of software and introduces the concept of "Software 3.0," where natural language processing becomes a new interface for programming.

### Summary of Key Points:

- **00:00 - Intro**: Karpathy introduces the topic of evolving software.
  
- **01:25 - Software evolution: From 1.0 to 3.0**: Acknowledges that software has undergone significant changes, leading to the current phase of Software 3.0.

- **04:40 - Programming in English: Rise of Software 3.0**: Discusses how programming is becoming more accessible with natural language, allowing users to interact with systems in English.

- **06:10 - LLMs as utilities, fabs, and operating systems**: Highlights the role of Large Language Models (LLMs) as 

✅ Great! You've successfully created an AI agent that uses OpenAI function calling with Klavis MCP servers to summarize YouTube videos!



## Case Study 2 : OpenAI + Gmail MCP Server (OAuth needed)

In [7]:
import webbrowser

gmail_mcp_server = klavis_client.mcp_server.create_server_instance(
    server_name=McpServerName.GMAIL,
    user_id="1234",
    platform_name="Klavis",
    connection_type=ConnectionType.STREAMABLE_HTTP,
)

webbrowser.open(gmail_mcp_server.oauth_url)

print(f"🔐 Opening OAuth authorization for Gmail, if you are not redirected, please open the following URL in your browser: {gmail_mcp_server.oauth_url}")


🔐 Opening OAuth authorization for Gmail, if you are not redirected, please open the following URL in your browser: https://api.klavis.ai/oauth/gmail/authorize?instance_id=d9d482b3-433a-4330-9a8b-9548c0b0a326


In [9]:
EMAIL_RECIPIENT = "zihaolin@klavis.ai" # Replace with your email
EMAIL_SUBJECT = "Test OpenAI + Gmail MCP Server"
EMAIL_BODY = "Hello World"

result = openai_with_mcp_server(
    mcp_server_url=gmail_mcp_server.server_url, 
    user_query=f"Please send an email to {EMAIL_RECIPIENT} with subject {EMAIL_SUBJECT} and body {EMAIL_BODY}"
)

print(result)

🔧 Calling: send_email, with args: {'to': ['zihaolin@klavis.ai'], 'subject': 'Test OpenAI + Gmail MCP Server', 'body': 'Hello World'}
The email has been successfully sent to zihaolin@klavis.ai with the subject "Test OpenAI + Gmail MCP Server" and the body "Hello World."


## Summary

This tutorial demonstrated how to integrate OpenAI's function calling capabilities with Klavis MCP servers to create powerful AI applications. We covered two practical examples:

**🎥 YouTube Integration**: Built an AI assistant that can automatically summarize YouTube videos by extracting transcripts and providing detailed, timestamped summaries.

**📧 Gmail Integration**: Created an AI-powered email assistant that can send emails through Gmail with OAuth authentication.

### Key Takeaways:
- **Easy Setup**: Klavis MCP servers can be created with just a few lines of code
- **OpenAI Compatible**: All tools are formatted for seamless OpenAI function calling
- **Versatile**: Support for both simple APIs (YouTube) and OAuth-authenticated services (Gmail)
- **Scalable**: The same pattern can be applied to any of the MCP servers available in Klavis

**Happy building!** 🚀
